In [1]:
import pandas as pd
import os

In [2]:
birth_rate_path = os.path.join("Birth_Rates.csv")
food_prices_path = os.path.join("Food_Prices.csv")
currency_path = os.path.join("currency.csv")

In [3]:
birth_rates = pd.read_csv(birth_rate_path)
food_prices = pd.read_csv(food_prices_path, encoding = "ISO-8859-1")
currency = pd.read_csv(currency_path)

In [4]:
#currency = currency.drop(['Unnamed: 0'], axis=1)
currency = currency.rename(columns={'0':'Date','1':'Rate','2':'Currency'})
currency

,Unnamed: 0,Date,Rate,Currency
0,0,Average,0.761312,AED UAE Dirham
1,0,"January 31, 1990",0.781105,AED UAE Dirham
2,0,"February 28, 1990",0.759321,AED UAE Dirham
3,0,"March 31, 1990",0.755618,AED UAE Dirham
4,0,"April 30, 1990",0.763662,AED UAE Dirham
...,...,...,...,...
21107,0,"November 30, 2019",0.68277,ZAR South African Rand
21108,0,"December 31, 2019",0.688739,ZAR South African Rand
21109,0,"January 08, 2020",0.694128,ZAR South African Rand
21110,0,Average,{{results.average}},ZAR South African Rand


In [5]:
currency = currency[currency.Date != "{{point.Date}}"]
currency = currency[currency.Date != "Average"]
currency = currency[currency.Rate != "{{point.Rate}}"]
currency = currency[currency.Rate != "{{results.average}}"]
currency

,Unnamed: 0,Date,Rate,Currency
1,0,"January 31, 1990",0.781105,AED UAE Dirham
2,0,"February 28, 1990",0.759321,AED UAE Dirham
3,0,"March 31, 1990",0.755618,AED UAE Dirham
4,0,"April 30, 1990",0.763662,AED UAE Dirham
5,0,"May 31, 1990",0.761055,AED UAE Dirham
...,...,...,...,...
21105,0,"September 30, 2019",0.680854,ZAR South African Rand
21106,0,"October 31, 2019",0.680027,ZAR South African Rand
21107,0,"November 30, 2019",0.68277,ZAR South African Rand
21108,0,"December 31, 2019",0.688739,ZAR South African Rand


In [8]:
birth_rates = birth_rates[['Country Name', '1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']]
birth_rates
birth_rates_2 = pd.DataFrame()
years = ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']
for index, row in birth_rates.iterrows():
    for year in years:
        birth_rates_2 = birth_rates_2.append([[row[0],year,row[year]]])

In [9]:
birth_rates_2

,0,1,2
0,Aruba,1992,18.446
0,Aruba,1993,17.859
0,Aruba,1994,17.270
0,Aruba,1995,16.691
0,Aruba,1996,16.132
...,...,...,...
0,Zimbabwe,2013,35.768
0,Zimbabwe,2014,34.978
0,Zimbabwe,2015,33.981
0,Zimbabwe,2016,32.864


In [25]:
currencies = currency['Currency'].unique()
currencies = pd.DataFrame(currencies)
currencies.to_csv('currencies.csv')